In [46]:
from io import BytesIO
import pycurl,timeit
from multiprocessing import Process
from multiprocessing import Manager

# Performance with proxy

In [47]:
def proxy_latency(prox_type,host,port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.PROXY, f"{host}:{port}")
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = prox_type
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    
    return m


# Performance without proxy

In [48]:
def normal_latency(port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = "N/A"
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    

# Setup

In [49]:
host = "130.64.148.78"
port = "8080"
range_k = 100



target = "https://github.com/"

# target = "http://www.cs.cmu.edu/~prs/bio.html"


# target = "http://www.cs.tufts.edu/comp/112/"

# Without proxy

## 1. One time

In [50]:
return_list = []
normal_latency(port,target,return_list)
print(return_list[0])

{'prox_type': 'N/A', 'target': 'https://github.com/', 'total-time': 0.168293, 'namelookup-time': 0.011801, 'connect-time': 0.032148, 'pretransfer-time': 0.060784, 'starttransfer-time': 0.089255}


## 2. Average time (100 request)

In [51]:
total = 0
return_list = []
for n in range(range_k):
    result = normal_latency(port,target,return_list)

for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.19488174999999985

# Our project - select + pthread

In [52]:
prox_type = "select"

## 1. One time without cache

In [53]:
return_list = []
proxy_latency(prox_type,host,port,target,return_list)
print(return_list[0])

{'prox_type': 'select', 'target': 'https://github.com/', 'total-time': 0.17127, 'namelookup-time': 5.3e-05, 'connect-time': 0.000182, 'pretransfer-time': 0.062683, 'starttransfer-time': 0.091231}


## 2. Average time with cache (100 reqests)

In [54]:
total = 0
return_list = []
for n in range(range_k):
    proxy_latency(prox_type,host,port,target,return_list)

    
for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.21362433999999986

# Fully fork version - without cache

In [55]:
# prox_type = "fork"


In [56]:
# total = 0

# for n in range(range_k):
#    result = proxy_latency(prox_type,host,port,target)
#    total+= result["total-time"]
    
# total/range_k

# Fully pthread version - without cache

In [57]:
# prox_type = "pthread"

In [58]:
# total = 0

# for n in range(range_k):
#    result = proxy_latency(prox_type,host,port,target)
#    total+= result["total-time"]
    
# total/range_k

# Simulate multi users - multiprocesses

In [59]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k    

0.7704979599999998

In [60]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k  

2.035939130000001

In [ ]:
# select+pthread 0.770 / 2.03

In [61]:
# select 0.730 / 3.057

0.23879620543016028